# Used Car Price Analysis

## Introduction
The goal of this project is to analyze factors that make a car more or less expensive and provide recommendations to a used car dealership. We will follow the CRISP-DM framework for this analysis.

## Data Understanding

In this section, we will perform an initial exploration of the dataset to understand its structure, characteristics, and any issues that need to be addressed.

### Load the Dataset
We will start by loading the dataset from the CSV file into a pandas DataFrame.


In [ ]:
# Importing Important libraries 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import scipy.stats as stats
import warnings

In [ ]:
# Load the CSV file
data = pd.read_csv('data/vehicles.csv', encoding='utf-8', na_values=['', 'NA', 'NaN'], keep_default_na=True)

#### Initial Inspection

We will inspect the first few rows of the dataset to get an overview of its structure and contents.

In [ ]:
# Display the last few rows of the dataset
data.tail()

#### Data Types and Summary Statistics

We will check the data types of each column and generate summary statistics to understand the distribution of numerical features.

In [ ]:
# Check data types of each column
data.info()

# Generate summary statistics for numerical features
data.describe()

#### Missing Values Analysis

We will identify and quantify missing values in each column to understand the extent of missing data.

In [ ]:
# Check for missing values in each column
missing_values_summary = data.isnull().sum()
missing_values_summary

In [ ]:
# We will perform initial visualizations and analyses to generate insights into the data.

# Ignore specific warning
warnings.filterwarnings("ignore", message=".*use_inf_as_na option is deprecated.*")

# Distribution of car prices
plt.figure(figsize=(10, 6))
sns.histplot(data['price'], bins=50, kde=True)
plt.title('Distribution of Car Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

# Price vs. Year
plt.figure(figsize=(10, 6))
sns.scatterplot(x='year', y='price', data=data)
plt.title('Price vs. Year')
plt.xlabel('Year')
plt.ylabel('Price')
plt.show()

#### Summary of Initial Findings

- **Data Structure:** The dataset contains various features related to used cars, such as price, year, manufacturer, model, condition, and mileage.

- **Missing Values:** Several columns have missing values that will need to be addressed in the data preparation phase.

- **Initial Insights:** Visualizations suggest that car prices are influenced by the car’s year and possibly other features.

This initial understanding of the data will guide the data preparation and modeling steps to follow.

## Data Preparation

In this section, we will clean the data by handling missing values, removing duplicates, and transforming the data into a suitable format for analysis and modeling.

### Handle Missing Values
We will start by deciding on a strategy to handle missing values in the dataset.


In [ ]:
# Display the summary of missing values
missing_values_summary = data.isnull().sum()

# Drop rows with too many missing values
threshold = 5  # Allow up to 5 missing values per row
data_cleaned = data.dropna(thresh=len(data.columns) - threshold).copy()

# Fill missing values where appropriate using .loc to avoid SettingWithCopyWarning
data_cleaned.loc[:, 'year'] = data_cleaned['year'].fillna(data_cleaned['year'].median())
data_cleaned.loc[:, 'odometer'] = data_cleaned['odometer'].fillna(data_cleaned['odometer'].median())
data_cleaned.loc[:, 'condition'] = data_cleaned['condition'].fillna('unknown')
data_cleaned.loc[:, 'cylinders'] = data_cleaned['cylinders'].fillna('unknown')
data_cleaned.loc[:, 'fuel'] = data_cleaned['fuel'].fillna('unknown')
data_cleaned.loc[:, 'title_status'] = data_cleaned['title_status'].fillna('unknown')
data_cleaned.loc[:, 'transmission'] = data_cleaned['transmission'].fillna('unknown')
data_cleaned.loc[:, 'drive'] = data_cleaned['drive'].fillna('unknown')
data_cleaned.loc[:, 'size'] = data_cleaned['size'].fillna('unknown')
data_cleaned.loc[:, 'type'] = data_cleaned['type'].fillna('unknown')
data_cleaned.loc[:, 'paint_color'] = data_cleaned['paint_color'].fillna('unknown')

# Verify the cleaning process
data_cleaned.info()

In [ ]:
# Droping VIN as it has many missing values and it is not crucial for analysis
data_cleaned = data_cleaned.drop(columns=['VIN'])

In [ ]:
#For manufacturer and model, we could drop rows with missing values or fill them with a placeholder
data_cleaned = data_cleaned.dropna(subset=['manufacturer', 'model'])

# Verifying dataframe again
data_cleaned.info()


In [ ]:
# Check for duplicates
duplicates = data_cleaned.duplicated().sum()
print(f'Number of duplicate rows: {duplicates}')

# Remove duplicates
data_cleaned = data_cleaned.drop_duplicates()

# Verify removal of duplicates
data_cleaned.info()

In [ ]:
#Convert Data Types
# We will ensure all columns have appropriate data types for analysis.

data_cleaned['year'] = data_cleaned['year'].astype(int)
data_cleaned['price'] = data_cleaned['price'].astype(float)
data_cleaned['odometer'] = data_cleaned['odometer'].astype(float)

# Verify data types
data_cleaned.dtypes

In [ ]:
# Creating a 'car_age' feature from the 'year' column
data_cleaned['car_age'] = 2024 - data_cleaned['year']

# Verify the new feature
data_cleaned[['year', 'car_age']].head()

#### Summary

- The DataFrame now has 372,156 entries with most columns having no missing values.
- Manufacturer and model columns have been cleaned by dropping rows with missing values.
- The VIN column has been dropped due to a high number of missing values.
- Created new feature such as **car_age** to enhance the analysis.

## Exploratory Data Analysis (EDA)

In this section, we will perform EDA to understand the distribution of key variables, identify relationships between variables, and detect outliers and anomalies.

### Handling Outliers

From the initial analysis of the `price` column, we found that a few outliers are significantly skewing the analysis. Removing these outliers will provide a clearer view of the data. We will identify and remove these outliers before proceeding with EDA.

In [ ]:
# Calculate IQR to identify outliers
Q1 = data_cleaned['price'].quantile(0.25)
Q3 = data_cleaned['price'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = data_cleaned[(data_cleaned['price'] >= lower_bound) & (data_cleaned['price'] <= upper_bound)]

# Verify the data after removing outliers
print(f"Original dataset size: {data_cleaned.shape}")
print(f"Dataset size after removing outliers: {df.shape}")

#### Distribution of Numerical Variables

We will start by visualizing the distribution of numerical variables such as price, year, and odometer using Plotly on the dataset

In [ ]:
fig = px.histogram(df, x='price', nbins=50, title='Distribution of Car Prices')
fig.update_layout(xaxis_title='Price', yaxis_title='Frequency')
fig.show()

In [ ]:
# Distribution of car year
fig = px.histogram(df, x='year', nbins=50, title='Distribution of Car Year')
fig.update_layout(xaxis_title='Year', yaxis_title='Frequency')
fig.show()

In [ ]:
# Distribution of odometer readings
fig = px.histogram(df, x='odometer', nbins=50, title='Distribution of Odometer Readings')
fig.update_layout(xaxis_title='Odometer', yaxis_title='Frequency')
fig.show()

#### Relationships Between Variables

We will explore relationships between variables using scatter plots and box plots in Plotly.


In [ ]:
# Price vs. Year
fig = px.scatter(df, x='year', y='price', title='Price vs. Year')
fig.update_layout(xaxis_title='Year', yaxis_title='Price')
fig.show()

In [ ]:
# Price vs. car age
fig = px.scatter(df, x='car_age', y='price', title='Price vs. car_age')
fig.update_layout(xaxis_title='car_age', yaxis_title='Price')
fig.show()

In [ ]:
# Price vs. Odometer
fig = px.scatter(df, x='odometer', y='price', title='Price vs. Odometer')
fig.update_layout(xaxis_title='Odometer', yaxis_title='Price')
fig.show()

In [ ]:
# Price by Manufacturer
fig = px.box(df, x='manufacturer', y='price', title='Price by Manufacturer')
fig.update_layout(xaxis_title='Manufacturer', yaxis_title='Price')
fig.show()

In [ ]:
# Select only numeric columns for correlation matrix
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Create and visualize the correlation matrix
fig = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    annotation_text=correlation_matrix.round(2).values,
    colorscale='Viridis'
)
fig.update_layout(title='Correlation Matrix', xaxis_title='Variables', yaxis_title='Variables')
fig.show()

#### Categorical Variable Analysis

We will analyze the distribution of categorical variables and their relationship with the target variable (price).


In [ ]:
# Price by Condition
fig = px.box(df, x='condition', y='price', title='Price by Condition')
fig.update_layout(xaxis_title='Condition', yaxis_title='Price')
fig.show()

In [ ]:
# Price by Fuel Type
fig = px.box(df, x='fuel', y='price', title='Price by Fuel Type')
fig.update_layout(xaxis_title='Fuel Type', yaxis_title='Price')
fig.show()

In [ ]:
# Price by Transmission Type
fig = px.box(df, x='transmission', y='price', title='Price by Transmission Type')
fig.update_layout(xaxis_title='Transmission', yaxis_title='Price')
fig.show()

In [ ]:
# Price by Drive Type
fig = px.box(df, x='drive', y='price', title='Price by Drive Type')
fig.update_layout(xaxis_title='Drive Type', yaxis_title='Price')
fig.show()

#### Summary of EDA

- **Distribution Analysis:** Visualized the distribution of key numerical variables (price, year, odometer). Removing outliers provided a clearer view of the price distribution.

- **Relationship Analysis:** Scatter plots revealed that newer cars tend to have higher prices, and cars with lower odometer readings are generally priced higher. Box plots showed significant price variation across different manufacturers.

- **Correlation Matrix:** Identified relationships between numerical variables using a correlation matrix.

- **Categorical Variable Analysis:** Box plots illustrated how car prices vary by condition, fuel type, transmission type, and drive type, providing insights into consumer preferences and market trends.

## Modeling

In this section, we will build and evaluate regression models to understand the factors that influence car prices. We will start with simple linear regression and explore more complex models. We will also use cross-validation to validate our models and perform hyperparameter tuning to optimize their performance.

Before diving into the modeling, we will preprocess the data to ensure it is in a suitable format for machine learning algorithms. This preprocessing includes steps such as one-hot encoding of categorical variables, scaling numerical features, and polynomial features.

#### Preprocessing

To prepare the data for modeling, we will perform the following preprocessing steps:

1. **One-Hot Encoding**: Convert categorical variables into a format that can be used by machine learning algorithms.
2. **Scaling**: Standardize numerical features to ensure they have a mean of 0 and a standard deviation of 1.
3. **Train-Test Split**: Split the data into training and testing sets.
4. **Transformations**: Apply any necessary transformations to the data.

In [ ]:
# Define features and target
features = ['year', 'odometer', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
target = 'price'

# Select the features and target
X = df[features]
y = df[target]


In [ ]:
# Define the preprocessing steps for numerical and categorical data
numerical_features = ['year', 'odometer']
categorical_features = [col for col in features if col not in numerical_features]


In [ ]:
# Create transformers for numerical and categorical features
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# Create a column transformer that applies the appropriate transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Simple Linear Regression

We will build and evaluate a simple linear regression model.

In [ ]:
# Create a preprocessing and modeling pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

pipeline

# Fit the linear regression model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Plot residuals vs. fitted values
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs. Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Q-Q plot of residuals
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.show()

### Insights from Residual Analysis

#### Residuals vs. Fitted Values Plot
- Funnel shape indicates heteroscedasticity, violating the constant variance assumption.
- Clear trend suggests missing key variables or need for a non-linear model.

#### Q-Q Plot of Residuals
- Deviation from the reference line, especially at the tails, indicates non-normality.
- Suggests presence of outliers or heavy-tailed data, affecting model reliability.

### Non Linear Regression
In this section, we will incorporate polynomial features of degree 3, use Ridge regression for feature selection, apply GridSearchCV for hyperparameter tuning, and then evaluate the model.


In [ ]:
# Define features and target
features = ['year', 'odometer', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
target = 'price'

# Select the features and target
X = df[features]
y = df[target]

# Define the preprocessing steps for numerical and categorical data
numerical_features = ['year', 'odometer']
categorical_features = [col for col in features if col not in numerical_features]

# Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer that applies the appropriate transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
# Create a Ridge regression model
ridge = Ridge(solver='auto')  # Using 'auto' solver for simplicity

# Create a pipeline with preprocessing and Ridge regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('ridge', ridge)
])


In [ ]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'ridge__alpha': np.logspace(-3, 2, 20) # Regularization strengths from 0.001 to 1000
}

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)


In [ ]:
# Best parameters from GridSearchCV
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')


In [ ]:
# Perform cross-validation
cv_scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='r2')

print(f'Cross-validated R^2 scores: {cv_scores}')
print(f'Mean cross-validated R^2 score: {cv_scores.mean()}')


In [ ]:
# Access the fitted preprocessor and extract feature names
fitted_preprocessor = grid_search.best_estimator_.named_steps['preprocessor']
fitted_numerical_transformer = fitted_preprocessor.named_transformers_['num'].named_steps['poly']
fitted_categorical_transformer = fitted_preprocessor.named_transformers_['cat']

# Extract feature names after fitting the preprocessor
poly_feature_names = fitted_numerical_transformer.get_feature_names_out(numerical_features)
cat_feature_names = fitted_categorical_transformer.get_feature_names_out(categorical_features)
feature_names = np.concatenate([poly_feature_names, cat_feature_names])

# Extract the best model and its coefficients
best_model = grid_search.best_estimator_.named_steps['ridge']
coefs = best_model.coef_

# Create a DataFrame for the coefficients
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefs})


In [ ]:
coef_df

In [ ]:
# Display the most important features
important_features = coef_df.sort_values(by='Coefficient', ascending=False).head(10)
print('Top 10 Important Features:')
print(important_features)

In [ ]:
# Plot the coefficients of the most important features
plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficient', y='Feature', data=important_features)
plt.title('Top 10 Important Features')
plt.xlabel('Coefficient')
plt.ylabel('Feature')
plt.show()

## Recommendations:
Based on the analysis, the following factors are key in determining car prices:

- Car Age: Newer cars tend to be priced higher.
- Odometer Reading: Cars with lower mileage are generally more expensive.
- Manufacturer and Model: Certain manufacturers and models, such as luxury and sports cars, have higher prices.
- Condition: Cars in better condition (e.g., ‘like new’, ‘excellent’) are valued more.
- Fuel Type: Gasoline and electric cars are generally priced higher than diesel and hybrid cars.
- Transmission and Drive Type: Manual transmission and all-wheel drive cars tend to be priced higher.

To fine-tune their inventory, used car dealerships should focus on acquiring newer cars with lower mileage, maintaining a diverse range of popular manufacturers and models, and ensuring that the cars are in good condition. Additionally, understanding consumer preferences for fuel type and transmission can help in pricing and marketing strategies.

## Future Analysis

Future analysis can provide deeper insights for the dealership. Detailed brand and model analysis can identify the most profitable models. Investigating geographic location impacts can reveal regional pricing trends. Analyzing seasonal trends will help plan inventory around peak buying times. Understanding consumer preferences through additional data sources can guide stock decisions. Implementing advanced machine learning models like Random Forest and Neural Networks can improve price prediction accuracy. A longitudinal study on depreciation rates will highlight cars that retain value better. Incorporating external economic indicators, such as fuel prices and interest rates, provides broader pricing context. Customer segmentation using clustering techniques enables targeted marketing. Sentiment analysis from reviews offers qualitative insights into customer satisfaction. Finally, predictive maintenance and warranty analysis can manage warranties and inform customers about future costs, enhancing trust and satisfaction.